##### Deep Learing ###

1) Data Cleaning

In [2]:
from logic.processing import load_data, preproc

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from gensim.models import Word2Vec

from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
2024-11-29 14:42:35.225329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = load_data('Combined Data.csv')

In [4]:
df = preproc(data,bi = False)

2) Split train/test

In [5]:
X = data['clean']
y = data['status']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

3) Tokenizer

In [12]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [19]:
np.array(X_train_token)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (42434,) + inhomogeneous part.

4) Balancing

In [17]:
ros = RandomOverSampler(random_state=101)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

5) Label encoding

In [27]:
y_train_encoded = pd.get_dummies(y_train_resampled).values
y_test_encoded = pd.get_dummies(y_test).values

5) Padding

In [28]:
max_len = 500

In [29]:
X_train_pad = pad_sequences(X_train_resampled, padding='post',maxlen=max_len)
X_test_pad = pad_sequences(X_test_token, padding='post',maxlen=max_len)

6.1) Model A

In [1]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

m0 = Sequential()
m0.add(Embedding(WORDS, DEPTH, input_length = LENGTH)) 
m0.add(Flatten())
m0.add(Dense(32, activation = 'relu'))
m0.add(Dense(3, activation = 'softmax'))
m0.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

2024-12-01 19:32:34.206134: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'WORDS' is not defined

In [ ]:
model.summary()

In [37]:
history = model.fit(
    X_train_pad, y_train_encoded,
    epochs=10,
    batch_size=64,
    verbose=1
)

Epoch 1/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 145s 212ms/step - accuracy: 0.3410 - loss: 1.6566
Epoch 2/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 134s 201ms/step - accuracy: 0.3358 - loss: 1.6428
Epoch 3/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 163s 246ms/step - accuracy: 0.3630 - loss: 1.6007
Epoch 4/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 154s 232ms/step - accuracy: 0.3842 - loss: 1.5727
Epoch 5/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 144s 217ms/step - accuracy: 0.5126 - loss: 1.3979
Epoch 6/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 167s 252ms/step - accuracy: 0.5218 - loss: 1.3685
Epoch 7/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 151s 227ms/step - accuracy: 0.5212 - loss: 1.3607
Epoch 8/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 145s 218ms/step - accuracy: 0.5229 - loss: 1.3455
Epoch 9/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 135s 204ms/step - accuracy: 0.5282 - loss: 1.3156
Epoch 10/10
664/664 ━━━━━━━━━━━━━━━━━━━━ 147s 222ms/step - accuracy: 0.5294 - loss: 1.3182


In [38]:
loss, accuracy = model.evaluate(X_test_pad, y_test_encoded, verbose=0)
print(f"Accuracy sur l'ensemble de test : {accuracy:.4f}")

Accuracy sur l'ensemble de test : 0.5394
